In [1]:
import pandas as pd
df = pd.read_csv("./sample.csv")

In [2]:
min_tweet = 1344836995441426432
tweet_inds = [int(link.split('/')[-1]) >= min_tweet for link in df['link']]
df = df[tweet_inds]

In [3]:
df = df.drop(columns = ['Unnamed: 0'])

In [4]:
import datetime
import time
clean_dates = []
for i in range(len(df)):
    original_date = df.iloc[i]['timestamp']
    day = int(original_date.split(' ')[2])
    (h,m,s) = original_date.split(' ')[3].split(':')
    month = 1
    if original_date.split(' ')[1] == "Jan":
        month = 1
    if original_date.split(' ')[1] == "Feb":
        month = 2
    if original_date.split(' ')[1] == "Mar":
        month = 3
    if original_date.split(' ')[1] == "Apr":
        month = 4
    
    ts = time.mktime(datetime.datetime(2021, month, day, int(h), int(m), int(s)).timetuple())
    clean_dates.append(ts)

In [6]:
df['datetime'] = list(df['timestamp'].values)
df['timestamp'] = clean_dates

In [7]:
df = df.rename(columns = {'bitcoin Returns': 'BTC_returns', 'doge Returns': 'DOGE_returns', 'tesla Returns': 'TSLA_returns', 'google Returns': 'GOOG_returns','apple Returns': 'AAPL_returns'})

In [8]:
df['GME_returns'] = [0 for i in range(len(df))]
df['SPY_returns'] = [0 for i in range(len(df))]

In [22]:
symbols = ['AAPL', 'BTC', 'DOGE', 'GME', 'GOOG', 'SPY', 'TSLA']
symbol_dat = {}
for sym in symbols:
    symbol_dat[sym] = pd.read_csv("scripts/processed_{:s}.csv".format(sym), header = None)

In [23]:
diff_window = 1
last_inds = {}
for sym in symbols:
    last_inds[sym] = 0

for col in df.columns:
    try:
        df[col] = df[col].astype(float)
    except:
        continue
df = df.sort_values('timestamp')
df = df.reset_index().drop(columns = ['index'])

for i in range(len(df)):
    ts = df.iloc[i]['timestamp']
    #print(i)
    for sym in symbols:
        price_change = 0
        
        for j in range(last_inds[sym], len(symbol_dat[sym])):
            if symbol_dat[sym].iloc[j+1][0] > ts:
                price_change = symbol_dat[sym].iloc[j+1][2] - symbol_dat[sym].iloc[j][2]
                last_inds[sym] = j
                break
        #print(price_change)
        df.at[i, '{:s}_returns'.format(sym)] = price_change.copy()

In [26]:
df.sort_values(['user', 'timestamp']).to_csv("./final_tweet_data")

,user,timestamp,tweet text,tweet sentiment,link,included symbols,BTC_returns,DOGE_returns,TSLA_returns,GOOG_returns,AAPL_returns,datetime,GME_returns,SPY_returns
0,JoeBiden,1.609491e+09,"In 2021, we’re going to build a brighter futur...",0.3818,twitter.com/JoeBiden/status/1344836995441426432,[],2753.11524,0.004930,1.79,0.00,-0.03,Fri Jan 01 02:45:00,0.00,0.25
1,JoeBiden,1.609494e+09,RT @NYRE: Here is the last interview for Presi...,0.0000,twitter.com/JoeBiden/status/1344850758068539393,[],2753.11524,0.004930,1.79,0.00,-0.03,Fri Jan 01 03:39:41,0.00,0.25
2,JoeBiden,1.609497e+09,Here’s to 2021. https://t.co/9HAQOSWgvO,0.0000,twitter.com/JoeBiden/status/1344862916194021377,[],2753.11524,0.004930,1.79,0.00,-0.03,Fri Jan 01 04:28:00,0.00,0.25
4,JoeBiden,1.609545e+09,Let me be clear: The Biden-Harris Administrati...,0.4019,twitter.com/JoeBiden/status/1345063487957917696,[],2753.11524,0.004930,1.79,0.00,-0.03,Fri Jan 01 17:45:00,0.00,0.25
5,JoeBiden,1.609637e+09,It’s the final weekend before the Georgia Sena...,0.0000,twitter.com/JoeBiden/status/1345449279964368896,[],654.75586,-0.000844,1.79,0.00,-0.03,Sat Jan 02 19:18:00,0.00,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,tim_cook,1.618980e+09,"Today’s verdict was just, but as Dr. King wrot...",0.0772,twitter.com/tim_cook/status/1384654106661310467,[],-2566.94141,-0.012575,-0.77,2.06,0.04,Tue Apr 20 23:43:59,1.00,0.68
469,tim_cook,1.619121e+09,Earth Day is a reminder that the fight for our...,-0.3818,twitter.com/tim_cook/status/1385243824390230018,['apple'],-668.62110,-0.012458,-0.45,-2.05,-0.01,Thu Apr 22 14:47:18,0.00,-0.16
507,tim_cook,1.619467e+09,"As we look toward recovery &amp; rebuilding, A...",0.3818,twitter.com/tim_cook/status/1386693881161392128,['apple'],1011.36328,0.001514,2.85,0.00,0.10,Mon Apr 26 14:49:19,-0.27,0.24
513,tim_cook,1.619487e+09,Amid a devastating rise of COVID cases in Indi...,-0.6486,twitter.com/tim_cook/status/1386780398609309698,['apple'],1011.36328,0.001514,-6.88,6.20,0.11,Mon Apr 26 20:33:06,-2.55,0.32


In [16]:
df['SPY_returns'].astype(float)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
568    0.0
569    0.0
570    0.0
571    0.0
572    1.0
Name: SPY_returns, Length: 573, dtype: float64

In [113]:
df

,user,timestamp,tweet text,tweet sentiment,link,included symbols,BTC_returns,DOGE_returns,TSLA_returns,GOOG_returns,AAPL_returns,datetime,GME_returns,SPY_returns
0,JoeBiden,1.609491e+09,"In 2021, we’re going to build a brighter futur...",0.3818,twitter.com/JoeBiden/status/1344836995441426432,[],-668.00000,0.000000,-3.00,7.00,0.00,Fri Jan 01 02:45:00,0.00,0.00
1,JoeBiden,1.609494e+09,RT @NYRE: Here is the last interview for Presi...,0.0000,twitter.com/JoeBiden/status/1344850758068539393,[],-668.00000,0.000000,-3.00,7.00,0.00,Fri Jan 01 03:39:41,0.00,0.00
2,JoeBiden,1.609497e+09,Here’s to 2021. https://t.co/9HAQOSWgvO,0.0000,twitter.com/JoeBiden/status/1344862916194021377,[],-668.00000,0.000000,-3.00,7.00,0.00,Fri Jan 01 04:28:00,0.00,0.00
3,tim_cook,1.609533e+09,Here’s to a New Year filled with promise and p...,0.6486,twitter.com/tim_cook/status/1345014435878866946,[],0.00000,0.000000,0.00,0.00,0.00,Fri Jan 01 14:30:05,0.00,0.00
4,JoeBiden,1.609545e+09,Let me be clear: The Biden-Harris Administrati...,0.4019,twitter.com/JoeBiden/status/1345063487957917696,[],-668.00000,0.000000,-3.00,7.00,0.00,Fri Jan 01 17:45:00,0.00,0.00
5,JoeBiden,1.609637e+09,It’s the final weekend before the Georgia Sena...,0.0000,twitter.com/JoeBiden/status/1345449279964368896,[],-668.00000,0.000000,0.00,0.00,0.00,Sat Jan 02 19:18:00,0.00,0.00
6,JoeBiden,1.609646e+09,"Mask up, folks.",0.0000,twitter.com/JoeBiden/status/1345490049727668226,[],-668.00000,0.000000,0.00,0.00,0.00,Sat Jan 02 22:00:00,0.00,0.00
7,JoeBiden,1.609659e+09,Floyd Little and I were students at Syracuse U...,0.0000,twitter.com/JoeBiden/status/1345542896662032384,[],-668.00000,0.000000,0.00,0.00,0.00,Sun Jan 03 01:30:00,0.00,0.00
8,JoeBiden,1.609718e+09,Here’s my promise to you: I’ll be a president ...,0.3182,twitter.com/JoeBiden/status/1345792290372116480,[],-668.00000,0.000000,0.00,-2.00,0.00,Sun Jan 03 18:01:00,0.00,0.00
9,JoeBiden,1.609731e+09,With just two days to go until the Georgia Sen...,-0.3400,twitter.com/JoeBiden/status/1345845695702396928,[],-668.00000,0.000000,0.00,0.00,0.00,Sun Jan 03 21:33:13,0.00,0.00
